## Анализ данных

### Задание: импорт данных

Возьмите данные по вызовам пожарных служб в Москве за 2015-2019 годы: https://video.ittensive.com/python-advanced/data-5283-2019-10-04.utf.csv Получите из них фрейм данных (таблицу значений). По этому фрейму вычислите среднее значение вызовов пожарных машин в месяц в одном округе Москвы, округлив до целых.

Примечание: найдите среднее значение вызовов, без учета года.

###############################################################################################################################

Импортируем pandas и считываем файл в датафрейм. Убираем колонку со значениями NaN.

In [1]:
import pandas as pd
df = pd.read_csv('https://video.ittensive.com/python-advanced/data-5283-2019-10-04.utf.csv', 
                  sep = ';', index_col = 'ID').drop(['Unnamed: 6'], axis = 1)
df.head()

,AdmArea,Year,global_id,Month,Calls
ID,,,,,
1,Центральный административный округ,2015,37153160,Январь,408
2,Северный административный округ,2015,37153161,Январь,408
3,Северо-Восточный административный округ,2015,37153162,Январь,445
4,Восточный административный округ,2015,37153163,Январь,619
5,Юго-Восточный административный округ,2015,37153164,Январь,541


Смотрим статистику по данным. Колонка с вызовами соответствует нужному типу данных.

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 627 entries, 1 to 627
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   AdmArea    627 non-null    object
 1   Year       627 non-null    int64 
 2   global_id  627 non-null    int64 
 3   Month      627 non-null    object
 4   Calls      627 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 29.4+ KB


Дропаем ненужные колонки и группируем данные по округам и месяцам.

In [3]:
df_groupby = df.drop(['Year', 'global_id'], axis = 1)
df_groupby.groupby(['AdmArea', 'Month']).mean()

Calls
AdmArea                          Month           
Восточный административный округ Август    690.75
                                 Апрель    801.40
                                 Декабрь   665.50
                                 Июль      574.00
                                 Июнь      635.60
...                                           ...
Южный административный округ     Февраль   613.50
                                 Январь    617.60
                                 август    445.00
                                 сентябрь  401.00
Южный административный округ     Февраль   617.00

[182 rows x 1 columns]

Получаем нужный нам ответ.

In [4]:
int(df.Calls.mean().round())

493

### Задание: данные из нескольких источников

Получите данные по безработице в Москве:

https://video.ittensive.com/python-advanced/data-9753-2019-07-25.utf.csv

Объедините эти данные индексами (Месяц/Год) с данными из предыдущего задания (вызовы пожарных) для Центральный административный округ:

https://video.ittensive.com/python-advanced/data-5283-2019-10-04.utf.csv

Найдите значение поля UnemployedMen в том месяце, когда было меньше всего вызовов в Центральном административном округе.

###############################################################################################################################

Импортируем библиотеки. Загружаем данные в датафрейм, приводим их к единому индексу и убираем колонки с NaN.

In [5]:
import pandas as pd

df_unemployed = pd.read_csv('https://video.ittensive.com/python-advanced/data-9753-2019-07-25.utf.csv', 
                            sep = ';').drop(['Unnamed: 9'], axis = 1)
df_unemployed.rename(columns = {'Period': 'Month'}, inplace = True)
df_unemployed = df_unemployed.set_index(['Month', 'Year'])
df_fireman = pd.read_csv('https://video.ittensive.com/python-advanced/data-5283-2019-10-04.utf.csv',
                          sep = ';', index_col = ['Month', 'Year']).drop(['ID', 'Unnamed: 6'], axis = 1)
print(df_fireman.head())
print(df_unemployed.head())

                                             AdmArea  global_id  Calls
Month  Year                                                           
Январь 2015       Центральный административный округ   37153160    408
       2015          Северный административный округ   37153161    408
       2015  Северо-Восточный административный округ   37153162    445
       2015         Восточный административный округ   37153163    619
       2015     Юго-Восточный административный округ   37153164    541
              global_id  ID  UnemployedMen  UnemployedWomen  UnemployedYoung  \
Month   Year                                                                   
Декабрь 2001   39809862   1           8896            25762             6696   
        2003   64592413   3           8679            24967             6226   
        2004   64592414   4           9822            25239             7318   
        2005   64592415   5          10844            23597             6890   
        2006   64592416

Соединяем датафреймы по индексу.

In [6]:
df = pd.merge(df_unemployed, df_fireman, left_index = True, right_index = True)
df.head()

global_id_x  ID  UnemployedMen  UnemployedWomen  UnemployedYoung  \
Month  Year                                                                     
Август 2015    273573620  58          15232            19966             5312   
       2015    273573620  58          15232            19966             5312   
       2015    273573620  58          15232            19966             5312   
       2015    273573620  58          15232            19966             5312   
       2015    273573620  58          15232            19966             5312   

             UnemployedDisabled  UnemployedTotal  \
Month  Year                                        
Август 2015                 996            35198   
       2015                 996            35198   
       2015                 996            35198   
       2015                 996            35198   
       2015                 996            35198   

                                             AdmArea  global_id_y  Calls  
Month  Year                                                               
Август 2015       Центральный административный округ     37153238    385  
       2015          Северный административный округ     37153239    374  
       2015  Северо-Восточный административный округ     37153240    461  
       2015         Восточный административный округ     37153241    600  
       2015     Юго-Восточный административный округ     37153242    507

Находим нужный нам ответ.

In [9]:
df_central = df[df['AdmArea'] == 'Центральный административный округ']
print(df_central.loc[df_central.Calls == df_central.Calls.min()]['UnemployedMen'].values)

[13465]


### Задание: выделение данных

Получите данные по безработице в Москве:

https://video.ittensive.com/python-advanced/data-9753-2019-07-25.utf.csv

Найдите, с какого года процент людей с ограниченными возможностями (UnemployedDisabled) среди всех безработных (UnemployedTotal) стал меньше 2%.

###############################################################################################################################

Импортируем pandas, загружаем данные в датафрейм, убираем колонку со значениями NaN.

In [4]:
import pandas as pd

df = pd.read_csv('https://video.ittensive.com/python-advanced/data-9753-2019-07-25.utf.csv', 
                  sep = ';', index_col = 'ID').drop(['Unnamed: 9'], axis = 1)
df.head()

,global_id,Year,UnemployedMen,UnemployedWomen,UnemployedYoung,UnemployedDisabled,UnemployedTotal,Period
ID,,,,,,,,
1,39809862,2001,8896,25762,6696,824,34658,Декабрь
3,64592413,2003,8679,24967,6226,1155,33646,Декабрь
4,64592414,2004,9822,25239,7318,1411,35061,Декабрь
5,64592415,2005,10844,23597,6890,1531,34441,Декабрь
6,64592416,2006,11041,22928,6604,1671,33969,Декабрь


Фильтруем данные по проценту людей с ограниченными возможностями среди всех безработных.

In [5]:
df[df['UnemployedDisabled'] < 0.02 * df['UnemployedTotal']]

,global_id,Year,UnemployedMen,UnemployedWomen,UnemployedYoung,UnemployedDisabled,UnemployedTotal,Period
ID,,,,,,,,
96,894075540,2018,10651,14043,2829,472,24694,Сентябрь
97,896740236,2018,10763,14375,2892,477,25138,Октябрь
98,904345746,2018,11080,14610,2911,467,25690,Ноябрь
99,909303094,2018,12063,15815,3077,527,27878,Декабрь
100,918869065,2019,12225,16078,2960,533,28303,Январь
101,918869066,2019,13123,16904,3103,575,30027,Февраль
102,918869067,2019,13433,17164,3102,593,30597,Март
103,926094627,2019,13803,17319,3060,593,31122,Апрель
104,933200250,2019,12818,16221,2711,577,29039,Май


Ответ: с сентября 2018 года.

### Задание: предсказание на 2020 год

Возьмите данные по безработице в городе Москва:

https://video.ittensive.com/python-advanced/data-9753-2019-07-25.utf.csv

Сгруппируйте данные по годам, и, если в году меньше 6 значений, отбросьте эти годы.
Постройте модель линейной регрессии по годам среднего значения отношения UnemployedDisabled к UnemployedTotal (процента людей с ограниченными возможностями) за месяц и ответьте, какое ожидается значение процента безработных инвалидов в 2020 году при сохранении текущей политики города Москвы?

Ответ округлите до сотых. Например, 2,32

###############################################################################################################################

Импортируем необходимые библиотеки.

In [25]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

Загружаем данные в датафрейм, создаём колонку с процентом людей с ограниченными возможностями от всех безработных.

In [26]:
data = pd.read_csv("https://video.ittensive.com/python-advanced/data-9753-2019-07-25.utf.csv", sep=";").dropna(axis = 1)
data['UnempDisabPerc'] = 100 * data["UnemployedDisabled"] / data["UnemployedTotal"]
data.head()

,global_id,ID,Year,UnemployedMen,UnemployedWomen,UnemployedYoung,UnemployedDisabled,UnemployedTotal,Period,UnempDisabPerc
0,39809862,1,2001,8896,25762,6696,824,34658,Декабрь,2.377517
1,64592413,3,2003,8679,24967,6226,1155,33646,Декабрь,3.432800
2,64592414,4,2004,9822,25239,7318,1411,35061,Декабрь,4.024415
3,64592415,5,2005,10844,23597,6890,1531,34441,Декабрь,4.445283
4,64592416,6,2006,11041,22928,6604,1671,33969,Декабрь,4.919191


Группируем данные по году, отфильтровывая их по количеству записей.

In [27]:
groupby_year = data.groupby('Year').filter(lambda x: x['UnempDisabPerc'].count() > 5)
groupby_year = groupby_year.groupby("Year").mean()
groupby_year.head()

,global_id,ID,UnemployedMen,UnemployedWomen,UnemployedYoung,UnemployedDisabled,UnemployedTotal,UnempDisabPerc
Year,,,,,,,,
2012,2.561585e+08,17.5,14505.333333,19315.25,4909.250000,1393.916667,33820.583333,4.122145
2013,2.735736e+08,30.5,10719.416667,15306.50,3388.666667,1047.416667,26025.916667,4.031303
2014,2.735736e+08,43.5,10271.666667,14361.50,3239.833333,990.500000,24633.166667,4.025994
2015,2.735736e+08,56.5,15142.666667,20273.25,5363.000000,1020.583333,35415.916667,2.903958
2016,2.763927e+08,69.5,18417.666667,23349.75,5856.916667,1205.166667,41767.416667,2.890474


Меняем размерность данных, приводя их к виду, в котором они могут быть использованы для обучения модели. Объявляем модель линейной регрессии, обучаем модель и делаем предсказание.

In [28]:
X = np.array(groupby_year.index).reshape(len(groupby_year.index), 1)
y = np.array(groupby_year['UnempDisabPerc']).reshape(len(groupby_year.index), 1)
model = LinearRegression()
model.fit(X, y)
print(np.round(model.predict(np.array(2020).reshape(1, 1)), 2))

[[1.52]]
